In [ ]:
import json
from multiprocessing import Pool
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm

def extract_id(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    result = []

    for item in data:
        if 'id' in item:
            result.append(item['id'])

    return result

def download_info(id):
    try:
        with webdriver.Chrome(options=chrome_options) as driver:
            driver.get(f'https://map.naver.com/p/search/%EB%8F%99%EB%AC%BC%EB%B3%91%EC%9B%90/place/{id}?c=13.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')
            driver.implicitly_wait(5)

            elements_to_click = [
                'div:nth-child(2) > div > div > a > span.rvCSr',
                'div.w9QyJ.vI8SM > div > span',
                'div:nth-child(9) > div > a > span.rvCSr > svg'
            ]

            for element_selector in elements_to_click:
                try:
                    elements = driver.find_elements(By.CSS_SELECTOR, element_selector)
                    if elements:
                        elements[0].click()
                        driver.implicitly_wait(5)

                except Exception as e:
                    print(f"Error: {e}")
                    pass

            hospital_info_element = []

            all_elements = driver.find_elements(By.CSS_SELECTOR, 'div:nth-child(2) > div.place_section_content > div *')
            for element in all_elements:
                hospital_info_element.append(element.text)

            return hospital_info_element

    except Exception as e:
        print(f"Error: {e}")
        return None

json_file_path = 'naver_total.json'
extracted_data = extract_id(json_file_path)

chrome_options = Options()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')

with Pool() as pool:
    results = list(tqdm(pool.imap(download_info, extracted_data), total=len(extracted_data), desc='Downloading info'))

hospital_info = [info for info in results if info is not None]

print(len(hospital_info))